In [4]:
import pandas as pd

AG = pd.read_csv('step04_FUHistory.csv')
AG = AG[AG['Year']<=2023]
champs = AG[AG['Round']==6]
champs.index = champs["Year"]

for year in champs['Year']:
    if champs.loc[year,'AFScore'] > champs.loc[year,'AUScore']:
        champs.loc[year,6] = champs.loc[year,'AFTeam']
    else:
        champs.loc[year,6] = champs.loc[year,'AUTeam']

# table of wins
fw = pd.crosstab(index=AG['AFTeam'],columns=AG['Year'])
uw = pd.crosstab(index=AG['AUTeam'],columns=AG['Year'])
chw = pd.crosstab(index=champs[6],columns=champs['Year'])

cw = fw.add(uw, fill_value=0).fillna(0).astype(int) - 1
cw = cw.add(chw, fill_value=0).fillna(0).astype(int)
cw = cw.replace(-1, '').astype(str)


C:\Users\Daddy\AppData\Local\Temp\ipykernel_13256\3235898579.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champs.loc[year,6] = champs.loc[year,'AUTeam']
